# Extract Transform & Load - Texas Hate Crime

In [1]:
#Import dependencies
import pandas as pd
from sqlalchemy import create_engine
from config import SQL_key

In [2]:
# Read csv file
path="./Raw_Resources/hate_crime.csv"
US_crime=pd.read_csv(path)

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# View current columns(Original)
US_crime.columns

Index(['INCIDENT_ID', 'DATA_YEAR', 'ORI', 'PUB_AGENCY_NAME', 'PUB_AGENCY_UNIT',
       'AGENCY_TYPE_NAME', 'STATE_ABBR', 'STATE_NAME', 'DIVISION_NAME',
       'REGION_NAME', 'POPULATION_GROUP_CODE', 'POPULATION_GROUP_DESC',
       'INCIDENT_DATE', 'ADULT_VICTIM_COUNT', 'JUVENILE_VICTIM_COUNT',
       'TOTAL_OFFENDER_COUNT', 'ADULT_OFFENDER_COUNT',
       'JUVENILE_OFFENDER_COUNT', 'OFFENDER_RACE', 'OFFENDER_ETHNICITY',
       'VICTIM_COUNT', 'OFFENSE_NAME', 'TOTAL_INDIVIDUAL_VICTIMS',
       'LOCATION_NAME', 'BIAS_DESC', 'VICTIM_TYPES', 'MULTIPLE_OFFENSE',
       'MULTIPLE_BIAS'],
      dtype='object')

In [4]:
US_crime.head()

INCIDENT_ID  DATA_YEAR        ORI PUB_AGENCY_NAME PUB_AGENCY_UNIT  \
0         3015       1991  AR0040200          Rogers             NaN   
1         3016       1991  AR0290100            Hope             NaN   
2           43       1991  AR0350100      Pine Bluff             NaN   
3           44       1991  AR0350100      Pine Bluff             NaN   
4         3017       1991  AR0350100      Pine Bluff             NaN   

  AGENCY_TYPE_NAME STATE_ABBR STATE_NAME       DIVISION_NAME REGION_NAME  ...  \
0             City         AR   Arkansas  West South Central       South  ...   
1             City         AR   Arkansas  West South Central       South  ...   
2             City         AR   Arkansas  West South Central       South  ...   
3             City         AR   Arkansas  West South Central       South  ...   
4             City         AR   Arkansas  West South Central       South  ...   

               OFFENDER_RACE OFFENDER_ETHNICITY VICTIM_COUNT  \
0                      White                NaN            1   
1  Black or African American                NaN            1   
2  Black or African American                NaN            1   
3  Black or African American                NaN            2   
4  Black or African American                NaN            1   

                                        OFFENSE_NAME  \
0                                       Intimidation   
1                                     Simple Assault   
2                                 Aggravated Assault   
3  Aggravated Assault;Destruction/Damage/Vandalis...   
4                                 Aggravated Assault   

   TOTAL_INDIVIDUAL_VICTIMS                       LOCATION_NAME  \
0                       1.0  Highway/Road/Alley/Street/Sidewalk   
1                       1.0  Highway/Road/Alley/Street/Sidewalk   
2                       1.0                      Residence/Home   
3                       1.0  Highway/Road/Alley/Street/Sidewalk   
4                       1.0                 Service/Gas Station   

                        BIAS_DESC  VICTIM_TYPES MULTIPLE_OFFENSE MULTIPLE_BIAS  
0  Anti-Black or African American    Individual                S             S  
1                      Anti-White    Individual                S             S  
2  Anti-Black or African American    Individual                S             S  
3                      Anti-White    Individual                M             S  
4                      Anti-White    Individual                S             S  

[5 rows x 28 columns]

In [5]:
#Select wanted columns and filter down to only the state of Texas
US_crime_df=US_crime[[ "INCIDENT_ID","DATA_YEAR","AGENCY_TYPE_NAME","STATE_NAME","POPULATION_GROUP_DESC","INCIDENT_DATE","TOTAL_OFFENDER_COUNT","OFFENDER_RACE","VICTIM_COUNT","OFFENSE_NAME","LOCATION_NAME","BIAS_DESC","VICTIM_TYPES"
]].copy()
Texas_crime=US_crime_df[US_crime['STATE_NAME']=='Texas']
Texas_crime.head()

INCIDENT_ID  DATA_YEAR AGENCY_TYPE_NAME STATE_NAME  \
4203         2540       1991           County      Texas   
4204         2411       1991             City      Texas   
4205         2412       1991             City      Texas   
4206         2413       1991             City      Texas   
4207         2414       1991             City      Texas   

                     POPULATION_GROUP_DESC INCIDENT_DATE  \
4203  MSA counties from 10,000 thru 24,999     01-SEP-91   
4204              Cities 1,000,000 or over     25-JAN-91   
4205              Cities 1,000,000 or over     26-JAN-91   
4206              Cities 1,000,000 or over     31-JAN-91   
4207              Cities 1,000,000 or over     24-FEB-91   

      TOTAL_OFFENDER_COUNT              OFFENDER_RACE  VICTIM_COUNT  \
4203                     1                      White             1   
4204                     3                      White             1   
4205                     1                      White             1   
4206                     3  Black or African American             1   
4207                     0                    Unknown             1   

                                  OFFENSE_NAME  \
4203                            Simple Assault   
4204                        Aggravated Assault   
4205                              Intimidation   
4206                        Aggravated Assault   
4207  Destruction/Damage/Vandalism of Property   

                                      LOCATION_NAME  \
4203  Jail/Prison/Penitentiary/Corrections Facility   
4204                        Parking/Drop Lot/Garage   
4205                              Convenience Store   
4206                                  Other/Unknown   
4207                                 Residence/Home   

                           BIAS_DESC VICTIM_TYPES  
4203  Anti-Black or African American   Individual  
4204  Anti-Black or African American   Individual  
4205                       Anti-Arab   Individual  
4206                      Anti-White   Individual  
4207  Anti-Black or African American        Other

In [6]:
#Break out leading offense to it's own column
Texas_crime["Parent Offense"]=Texas_crime["OFFENSE_NAME"].str.split("/|;", n=1, expand=True)[0]
Texas_crime.head()

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


INCIDENT_ID  DATA_YEAR AGENCY_TYPE_NAME STATE_NAME  \
4203         2540       1991           County      Texas   
4204         2411       1991             City      Texas   
4205         2412       1991             City      Texas   
4206         2413       1991             City      Texas   
4207         2414       1991             City      Texas   

                     POPULATION_GROUP_DESC INCIDENT_DATE  \
4203  MSA counties from 10,000 thru 24,999     01-SEP-91   
4204              Cities 1,000,000 or over     25-JAN-91   
4205              Cities 1,000,000 or over     26-JAN-91   
4206              Cities 1,000,000 or over     31-JAN-91   
4207              Cities 1,000,000 or over     24-FEB-91   

      TOTAL_OFFENDER_COUNT              OFFENDER_RACE  VICTIM_COUNT  \
4203                     1                      White             1   
4204                     3                      White             1   
4205                     1                      White             1   
4206                     3  Black or African American             1   
4207                     0                    Unknown             1   

                                  OFFENSE_NAME  \
4203                            Simple Assault   
4204                        Aggravated Assault   
4205                              Intimidation   
4206                        Aggravated Assault   
4207  Destruction/Damage/Vandalism of Property   

                                      LOCATION_NAME  \
4203  Jail/Prison/Penitentiary/Corrections Facility   
4204                        Parking/Drop Lot/Garage   
4205                              Convenience Store   
4206                                  Other/Unknown   
4207                                 Residence/Home   

                           BIAS_DESC VICTIM_TYPES      Parent Offense  
4203  Anti-Black or African American   Individual      Simple Assault  
4204  Anti-Black or African American   Individual  Aggravated Assault  
4205                       Anti-Arab   Individual        Intimidation  
4206                      Anti-White   Individual  Aggravated Assault  
4207  Anti-Black or African American        Other         Destruction

In [7]:
#Parse out day of week column from incident date column
Texas_crime['INCIDENT_DATE'] = pd.to_datetime(Texas_crime['INCIDENT_DATE'])
Texas_crime['Day of Week'] = Texas_crime['INCIDENT_DATE'].dt.day_name()
Texas_crime.head()

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


INCIDENT_ID  DATA_YEAR AGENCY_TYPE_NAME STATE_NAME  \
4203         2540       1991           County      Texas   
4204         2411       1991             City      Texas   
4205         2412       1991             City      Texas   
4206         2413       1991             City      Texas   
4207         2414       1991             City      Texas   

                     POPULATION_GROUP_DESC INCIDENT_DATE  \
4203  MSA counties from 10,000 thru 24,999    1991-09-01   
4204              Cities 1,000,000 or over    1991-01-25   
4205              Cities 1,000,000 or over    1991-01-26   
4206              Cities 1,000,000 or over    1991-01-31   
4207              Cities 1,000,000 or over    1991-02-24   

      TOTAL_OFFENDER_COUNT              OFFENDER_RACE  VICTIM_COUNT  \
4203                     1                      White             1   
4204                     3                      White             1   
4205                     1                      White             1   
4206                     3  Black or African American             1   
4207                     0                    Unknown             1   

                                  OFFENSE_NAME  \
4203                            Simple Assault   
4204                        Aggravated Assault   
4205                              Intimidation   
4206                        Aggravated Assault   
4207  Destruction/Damage/Vandalism of Property   

                                      LOCATION_NAME  \
4203  Jail/Prison/Penitentiary/Corrections Facility   
4204                        Parking/Drop Lot/Garage   
4205                              Convenience Store   
4206                                  Other/Unknown   
4207                                 Residence/Home   

                           BIAS_DESC VICTIM_TYPES      Parent Offense  \
4203  Anti-Black or African American   Individual      Simple Assault   
4204  Anti-Black or African American   Individual  Aggravated Assault   
4205                       Anti-Arab   Individual        Intimidation   
4206                      Anti-White   Individual  Aggravated Assault   
4207  Anti-Black or African American        Other         Destruction   

     Day of Week  
4203      Sunday  
4204      Friday  
4205    Saturday  
4206    Thursday  
4207      Sunday

In [8]:
# Use datetime to get month and day from incident date
Texas_crime['Month'] = Texas_crime['INCIDENT_DATE'].dt.month
Texas_crime['Day'] = Texas_crime['INCIDENT_DATE'].dt.day
Texas_crime.head()

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


INCIDENT_ID  DATA_YEAR AGENCY_TYPE_NAME STATE_NAME  \
4203         2540       1991           County      Texas   
4204         2411       1991             City      Texas   
4205         2412       1991             City      Texas   
4206         2413       1991             City      Texas   
4207         2414       1991             City      Texas   

                     POPULATION_GROUP_DESC INCIDENT_DATE  \
4203  MSA counties from 10,000 thru 24,999    1991-09-01   
4204              Cities 1,000,000 or over    1991-01-25   
4205              Cities 1,000,000 or over    1991-01-26   
4206              Cities 1,000,000 or over    1991-01-31   
4207              Cities 1,000,000 or over    1991-02-24   

      TOTAL_OFFENDER_COUNT              OFFENDER_RACE  VICTIM_COUNT  \
4203                     1                      White             1   
4204                     3                      White             1   
4205                     1                      White             1   
4206                     3  Black or African American             1   
4207                     0                    Unknown             1   

                                  OFFENSE_NAME  \
4203                            Simple Assault   
4204                        Aggravated Assault   
4205                              Intimidation   
4206                        Aggravated Assault   
4207  Destruction/Damage/Vandalism of Property   

                                      LOCATION_NAME  \
4203  Jail/Prison/Penitentiary/Corrections Facility   
4204                        Parking/Drop Lot/Garage   
4205                              Convenience Store   
4206                                  Other/Unknown   
4207                                 Residence/Home   

                           BIAS_DESC VICTIM_TYPES      Parent Offense  \
4203  Anti-Black or African American   Individual      Simple Assault   
4204  Anti-Black or African American   Individual  Aggravated Assault   
4205                       Anti-Arab   Individual        Intimidation   
4206                      Anti-White   Individual  Aggravated Assault   
4207  Anti-Black or African American        Other         Destruction   

     Day of Week  Month  Day  
4203      Sunday      9    1  
4204      Friday      1   25  
4205    Saturday      1   26  
4206    Thursday      1   31  
4207      Sunday      2   24

In [9]:
#Rename columns
Texas_crime = Texas_crime.rename(columns = {'INCIDENT_ID':'Incident_ID', 
                                            'DATA_YEAR':'Year', 
                                            'AGENCY_TYPE_NAME': 'Reporting_Agency',
                                            'STATE_NAME':'State', 
                                            'POPULATION_GROUP_DESC': 'Population_Group',
                                            'INCIDENT_DATE':'Incident_Date', 
                                            'TOTAL_OFFENDER_COUNT': 'Offender_Count',
                                            'OFFENDER_RACE': 'Offender_Race', 
                                            'VICTIM_COUNT':'Victim_Count',
                                            'OFFENSE_NAME':'Offense_Name',
                                            'LOCATION_NAME':'Location', 
                                            'BIAS_DESC':'Bias',
                                            'VICTIM_TYPES':'Victim_Type',
                                           "Parent Offense":"Parent_Offense",
                                           "Day of Week":"Day_of_Week"})
# View updated Texas csv
Texas_crime.head()

Incident_ID  Year Reporting_Agency  State  \
4203         2540  1991           County  Texas   
4204         2411  1991             City  Texas   
4205         2412  1991             City  Texas   
4206         2413  1991             City  Texas   
4207         2414  1991             City  Texas   

                          Population_Group Incident_Date  Offender_Count  \
4203  MSA counties from 10,000 thru 24,999    1991-09-01               1   
4204              Cities 1,000,000 or over    1991-01-25               3   
4205              Cities 1,000,000 or over    1991-01-26               1   
4206              Cities 1,000,000 or over    1991-01-31               3   
4207              Cities 1,000,000 or over    1991-02-24               0   

                  Offender_Race  Victim_Count  \
4203                      White             1   
4204                      White             1   
4205                      White             1   
4206  Black or African American             1   
4207                    Unknown             1   

                                  Offense_Name  \
4203                            Simple Assault   
4204                        Aggravated Assault   
4205                              Intimidation   
4206                        Aggravated Assault   
4207  Destruction/Damage/Vandalism of Property   

                                           Location  \
4203  Jail/Prison/Penitentiary/Corrections Facility   
4204                        Parking/Drop Lot/Garage   
4205                              Convenience Store   
4206                                  Other/Unknown   
4207                                 Residence/Home   

                                Bias Victim_Type      Parent_Offense  \
4203  Anti-Black or African American  Individual      Simple Assault   
4204  Anti-Black or African American  Individual  Aggravated Assault   
4205                       Anti-Arab  Individual        Intimidation   
4206                      Anti-White  Individual  Aggravated Assault   
4207  Anti-Black or African American       Other         Destruction   

     Day_of_Week  Month  Day  
4203      Sunday      9    1  
4204      Friday      1   25  
4205    Saturday      1   26  
4206    Thursday      1   31  
4207      Sunday      2   24

In [10]:
print(len(Texas_crime.columns))
Texas_crime.columns

17


Index(['Incident_ID', 'Year', 'Reporting_Agency', 'State', 'Population_Group',
       'Incident_Date', 'Offender_Count', 'Offender_Race', 'Victim_Count',
       'Offense_Name', 'Location', 'Bias', 'Victim_Type', 'Parent_Offense',
       'Day_of_Week', 'Month', 'Day'],
      dtype='object')

In [11]:
Texas_crime["Parent_Offense"] = Texas_crime["Parent_Offense"].replace(
                                                    {"Robbery":"Burglary",
                                                    "Shoplifting":"Burglary",
                                                    "Pocket-picking":"Burglary",
                                                    "Theft From Building":"Burglary",
                                                    "Theft From Coin-Operated Machine or Device":"Burglary",
                                                    "Sexual Assault with An Object":"Sexual Assault",
                                                    "Rape":"Sexual Assault",
                                                    "Fondling":"Sexual Assault"})

In [12]:
# Push transformed CSV file to folder
Texas_crime.to_csv("Transformed_Resources/texas_hatecrime.csv", index=False, header=True)

In [13]:
# Create Postgress connection
rds_connection_string = f'postgres:{SQL_key}@localhost:5432/hate_crimes_db'
engine = create_engine(f'postgresql://{rds_connection_string}')

In [14]:
# View tables
engine.table_names()

['Texas_Hate_Crimes', 'Austin_Hate_Crimes']

In [15]:
# Push transformed file to Postgres
Texas_crime.to_sql(name="Texas_Hate_Crimes", con=engine, if_exists='append', index=False)

In [16]:
# Query file from Postgres
pd.read_sql_query('select * from "Texas_Hate_Crimes"', con=engine).head()

Incident_ID Incident_Date  Year  Day  Month Day_of_Week Reporting_Agency  \
0         2540    1991-09-01  1991    1      9      Sunday           County   
1         2411    1991-01-25  1991   25      1      Friday             City   
2         2412    1991-01-26  1991   26      1    Saturday             City   
3         2413    1991-01-31  1991   31      1    Thursday             City   
4         2414    1991-02-24  1991   24      2      Sunday             City   

                               Offense_Name  Offender_Count  \
0                            Simple Assault               1   
1                        Aggravated Assault               3   
2                              Intimidation               1   
3                        Aggravated Assault               3   
4  Destruction/Damage/Vandalism of Property               0   

               Offender_Race  Victim_Count  \
0                      White             1   
1                      White             1   
2                      White             1   
3  Black or African American             1   
4                    Unknown             1   

                                        Location  \
0  Jail/Prison/Penitentiary/Corrections Facility   
1                        Parking/Drop Lot/Garage   
2                              Convenience Store   
3                                  Other/Unknown   
4                                 Residence/Home   

                             Bias Victim_Type  State  \
0  Anti-Black or African American  Individual  Texas   
1  Anti-Black or African American  Individual  Texas   
2                       Anti-Arab  Individual  Texas   
3                      Anti-White  Individual  Texas   
4  Anti-Black or African American       Other  Texas   

                       Population_Group      Parent_Offense  
0  MSA counties from 10,000 thru 24,999      Simple Assault  
1              Cities 1,000,000 or over  Aggravated Assault  
2              Cities 1,000,000 or over        Intimidation  
3              Cities 1,000,000 or over  Aggravated Assault  
4              Cities 1,000,000 or over         Destruction

# Extract Transform & Load - Austin Hate Crime

In [17]:
# Bring in hate crime csvs
austin2017_csv = "Raw_Resources/Hate_Crimes_2017.csv"
austin2018_csv = "Raw_Resources/Hate_Crimes_2018_Final.csv"

In [18]:
# Read csv files
austin2017_df = pd.read_csv(austin2017_csv)
austin2018_df = pd.read_csv(austin2018_csv)

In [19]:
austin2017_df.head()

Month Incident Number Date of Incident/Day of Week  \
0   January     2017-241137               01/01/2017/Sun   
1  February     2017-580344               02/01/2017/Wed   
2     March     2017-800291              03/21/2017/Tues   
3     April    2017-1021534               04/12/2017/Wed   
4       May    2017-1351550               05/15/2017/Mon   

   Number of vitims under 18  Number of Victims Over 18  \
0                          0                          1   
1                          0                          1   
2                          0                          0   
3                          0                          0   
4                          1                          0   

   Number of Offenders under 18  Number of Offenders over 18  \
0                             0                            1   
1                             0                            1   
2                             0                            0   
3                             0                            0   
4                             1                            2   

               Race or Ethnic of Offender             Offense  \
0                      White/Not Hispanic  Aggravated Assault   
1  Black or African American/Not Hispanic  Aggravated Assault   
2                                 Unknown         Destruction   
3                           White/Unknown      Simple Assault   
4                      White/Not Hispanic      Simple Assault   

                     Offense Location                            Bias  \
0                     Park/Playground  Anti-Black or African American   
1  Highway/Road/Alley/Street/Sidewalk                      Anti-White   
2  Highway/Road/Alley/Street/Sidewalk                     Anti-Jewish   
3              Air/Bus/Train Terminal                     Anti-Jewish   
4                      Residence/Home                 Anti-Gay (Male)   

  Victim Type  
0  Individual  
1  Individual  
2       Other  
3  Individual  
4  Individual

In [20]:
austin2018_df.head()

Month Incident Number Date of Incident/Day of Week  \
0   January     2018-251458              01 25 2018/Thur   
1   January     2018-200595               01 19 2018/Fri   
2  February     2018-400447              02 08 2018/Thur   
3  February     2018-530804              02 22 2018/Thur   
4     March     2018-611809               03 02 2018/Fri   

   Number of vitims under 18  Number of Victims Over 18  \
0                          0                          2   
1                          0                          0   
2                          0                          1   
3                          0                          1   
4                          0                          1   

   Number of Offenders under 18  Number of Offenders over 18  \
0                             0                            1   
1                             0                            0   
2                             0                            1   
3                             0                            1   
4                             0                            4   

  Race/Ethnic of Offender(s)        Offense(s)     Offense Location  \
0          White/NotHispanic  Burglary/Assault       Residence/Home   
1                    Unknown         Vandalism   Parking Lot/Garage   
2          White/NotHispanic           Assault   Parking Lot/Garage   
3          White/NotHispanic         Vandalism  Highway/Road/Street   
4              Black/Unknown           Assault  Highway/Road/Street   

            Bias Victim Type  
0   Anti-Lesbian  Individual  
1     Anti-Black     Vehicle  
2       Anti-Gay  Individual  
3     Anti-Black  Individual  
4  Anti-Hispanic  Individual

In [21]:
#Make column name changes before combining the two files
austin2018_rename_df = austin2018_df.rename(columns={"Offense(s)": "Offense","Race/Ethnic of Offender(s)":"Race or Ethnic of Offender"})
austin2018_rename_df.head()

Month Incident Number Date of Incident/Day of Week  \
0   January     2018-251458              01 25 2018/Thur   
1   January     2018-200595               01 19 2018/Fri   
2  February     2018-400447              02 08 2018/Thur   
3  February     2018-530804              02 22 2018/Thur   
4     March     2018-611809               03 02 2018/Fri   

   Number of vitims under 18  Number of Victims Over 18  \
0                          0                          2   
1                          0                          0   
2                          0                          1   
3                          0                          1   
4                          0                          1   

   Number of Offenders under 18  Number of Offenders over 18  \
0                             0                            1   
1                             0                            0   
2                             0                            1   
3                             0                            1   
4                             0                            4   

  Race or Ethnic of Offender           Offense     Offense Location  \
0          White/NotHispanic  Burglary/Assault       Residence/Home   
1                    Unknown         Vandalism   Parking Lot/Garage   
2          White/NotHispanic           Assault   Parking Lot/Garage   
3          White/NotHispanic         Vandalism  Highway/Road/Street   
4              Black/Unknown           Assault  Highway/Road/Street   

            Bias Victim Type  
0   Anti-Lesbian  Individual  
1     Anti-Black     Vehicle  
2       Anti-Gay  Individual  
3     Anti-Black  Individual  
4  Anti-Hispanic  Individual

In [22]:
#Combining the 2018 and 2017 Austin data together
austin_combinded_df = pd.concat([austin2017_df, austin2018_rename_df], sort=False)
austin_combinded_df.head()

Month Incident Number Date of Incident/Day of Week  \
0   January     2017-241137               01/01/2017/Sun   
1  February     2017-580344               02/01/2017/Wed   
2     March     2017-800291              03/21/2017/Tues   
3     April    2017-1021534               04/12/2017/Wed   
4       May    2017-1351550               05/15/2017/Mon   

   Number of vitims under 18  Number of Victims Over 18  \
0                          0                          1   
1                          0                          1   
2                          0                          0   
3                          0                          0   
4                          1                          0   

   Number of Offenders under 18  Number of Offenders over 18  \
0                             0                            1   
1                             0                            1   
2                             0                            0   
3                             0                            0   
4                             1                            2   

               Race or Ethnic of Offender             Offense  \
0                      White/Not Hispanic  Aggravated Assault   
1  Black or African American/Not Hispanic  Aggravated Assault   
2                                 Unknown         Destruction   
3                           White/Unknown      Simple Assault   
4                      White/Not Hispanic      Simple Assault   

                     Offense Location                            Bias  \
0                     Park/Playground  Anti-Black or African American   
1  Highway/Road/Alley/Street/Sidewalk                      Anti-White   
2  Highway/Road/Alley/Street/Sidewalk                     Anti-Jewish   
3              Air/Bus/Train Terminal                     Anti-Jewish   
4                      Residence/Home                 Anti-Gay (Male)   

  Victim Type  
0  Individual  
1  Individual  
2       Other  
3  Individual  
4  Individual

In [23]:
#Renaming df to allow for combined code
Austin_crime=austin_combinded_df

In [24]:
#2017 and 2018 date format are different so breaking them up so we can match them. (remove day of week)
Austin_crime["Date of Incident/Day of Week"]=Austin_crime["Date of Incident/Day of Week"].str.rpartition("/", expand=True)
Austin_crime.head(20)

Month Incident Number Date of Incident/Day of Week  \
0     January     2017-241137                   01/01/2017   
1    February     2017-580344                   02/01/2017   
2       March     2017-800291                   03/21/2017   
3       April    2017-1021534                   04/12/2017   
4         May    2017-1351550                   05/15/2017   
5        June    2017-1550219                   06/04/2017   
6        June    2017-1660914                    6/15/2017   
7        June    2017-1681488                   06/17/2017   
8        July    2017-1820275                   07/01/2017   
9       July     2017-1862526                   07/05/2017   
10      July     2017-2011198                   07/20/2017   
11      July     2017-2020124                    7/21/2017   
12    October    2017-2880227                   10/15/2017   
13    October    2017-2971392                   10/24/2017   
14  November     2017-3141125                   11/10/2017   
15  November     2017-3200108                   11/16/2017   
16  November     2017-5043049                   11/26/2017   
0     January     2018-251458                   01 25 2018   
1     January     2018-200595                   01 19 2018   
2    February     2018-400447                   02 08 2018   

    Number of vitims under 18  Number of Victims Over 18  \
0                           0                          1   
1                           0                          1   
2                           0                          0   
3                           0                          0   
4                           1                          0   
5                           0                          1   
6                           0                          1   
7                           0                          1   
8                           0                          1   
9                           0                          1   
10                          0                          1   
11                          0                          1   
12                          0                          1   
13                          0                          1   
14                          0                          1   
15                          0                          1   
16                          0                          1   
0                           0                          2   
1                           0                          0   
2                           0                          1   

    Number of Offenders under 18  Number of Offenders over 18  \
0                              0                            1   
1                              0                            1   
2                              0                            0   
3                              0                            0   
4                              1                            2   
5                              0                            2   
6                              0                            1   
7                              0                            1   
8                              0                            1   
9                              0                            1   
10                             0                            1   
11                             0                            1   
12                             0                            1   
13                             2                            0   
14                             0                            1   
15                             0                            1   
16                             0                            0   
0                              0                            1   
1                              0                            0   
2                              0                            1   

                Race or Ethnic of Offender             Offense  \
0           

In [25]:
#Format dates so they match across the column
Austin_crime["Date of Incident/Day of Week"] = Austin_crime["Date of Incident/Day of Week"].str.strip()
Austin_crime["Date of Incident/Day of Week"] = Austin_crime["Date of Incident/Day of Week"].str.replace(" ","/")
Austin_crime.head(20)

Month Incident Number Date of Incident/Day of Week  \
0     January     2017-241137                   01/01/2017   
1    February     2017-580344                   02/01/2017   
2       March     2017-800291                   03/21/2017   
3       April    2017-1021534                   04/12/2017   
4         May    2017-1351550                   05/15/2017   
5        June    2017-1550219                   06/04/2017   
6        June    2017-1660914                    6/15/2017   
7        June    2017-1681488                   06/17/2017   
8        July    2017-1820275                   07/01/2017   
9       July     2017-1862526                   07/05/2017   
10      July     2017-2011198                   07/20/2017   
11      July     2017-2020124                    7/21/2017   
12    October    2017-2880227                   10/15/2017   
13    October    2017-2971392                   10/24/2017   
14  November     2017-3141125                   11/10/2017   
15  November     2017-3200108                   11/16/2017   
16  November     2017-5043049                   11/26/2017   
0     January     2018-251458                   01/25/2018   
1     January     2018-200595                   01/19/2018   
2    February     2018-400447                   02/08/2018   

    Number of vitims under 18  Number of Victims Over 18  \
0                           0                          1   
1                           0                          1   
2                           0                          0   
3                           0                          0   
4                           1                          0   
5                           0                          1   
6                           0                          1   
7                           0                          1   
8                           0                          1   
9                           0                          1   
10                          0                          1   
11                          0                          1   
12                          0                          1   
13                          0                          1   
14                          0                          1   
15                          0                          1   
16                          0                          1   
0                           0                          2   
1                           0                          0   
2                           0                          1   

    Number of Offenders under 18  Number of Offenders over 18  \
0                              0                            1   
1                              0                            1   
2                              0                            0   
3                              0                            0   
4                              1                            2   
5                              0                            2   
6                              0                            1   
7                              0                            1   
8                              0                            1   
9                              0                            1   
10                             0                            1   
11                             0                            1   
12                             0                            1   
13                             2                            0   
14                             0                            1   
15                             0                            1   
16                             0                            0   
0                              0                            1   
1                              0                            0   
2                              0                            1   

                Race or Ethnic of Offender             Offense  \
0           

In [26]:
#Add Day of Week option
Austin_crime['Date of Incident/Day of Week'] = pd.to_datetime(Austin_crime['Date of Incident/Day of Week'])
Austin_crime['Day of Week'] = Austin_crime['Date of Incident/Day of Week'].dt.day_name()
Austin_crime.head()

Month Incident Number Date of Incident/Day of Week  \
0   January     2017-241137                   2017-01-01   
1  February     2017-580344                   2017-02-01   
2     March     2017-800291                   2017-03-21   
3     April    2017-1021534                   2017-04-12   
4       May    2017-1351550                   2017-05-15   

   Number of vitims under 18  Number of Victims Over 18  \
0                          0                          1   
1                          0                          1   
2                          0                          0   
3                          0                          0   
4                          1                          0   

   Number of Offenders under 18  Number of Offenders over 18  \
0                             0                            1   
1                             0                            1   
2                             0                            0   
3                             0                            0   
4                             1                            2   

               Race or Ethnic of Offender             Offense  \
0                      White/Not Hispanic  Aggravated Assault   
1  Black or African American/Not Hispanic  Aggravated Assault   
2                                 Unknown         Destruction   
3                           White/Unknown      Simple Assault   
4                      White/Not Hispanic      Simple Assault   

                     Offense Location                            Bias  \
0                     Park/Playground  Anti-Black or African American   
1  Highway/Road/Alley/Street/Sidewalk                      Anti-White   
2  Highway/Road/Alley/Street/Sidewalk                     Anti-Jewish   
3              Air/Bus/Train Terminal                     Anti-Jewish   
4                      Residence/Home                 Anti-Gay (Male)   

  Victim Type Day of Week  
0  Individual      Sunday  
1  Individual   Wednesday  
2       Other     Tuesday  
3  Individual   Wednesday  
4  Individual      Monday

In [27]:
#Combine the Victim totals and drop the break out columns
Austin_crime["Total Victims"] = Austin_crime["Number of vitims under 18"] + Austin_crime["Number of Victims Over 18"]
Austin_crime = Austin_crime.drop(['Number of vitims under 18', 'Number of Victims Over 18'], axis=1)

In [28]:
Austin_crime.head()

Month Incident Number Date of Incident/Day of Week  \
0   January     2017-241137                   2017-01-01   
1  February     2017-580344                   2017-02-01   
2     March     2017-800291                   2017-03-21   
3     April    2017-1021534                   2017-04-12   
4       May    2017-1351550                   2017-05-15   

   Number of Offenders under 18  Number of Offenders over 18  \
0                             0                            1   
1                             0                            1   
2                             0                            0   
3                             0                            0   
4                             1                            2   

               Race or Ethnic of Offender             Offense  \
0                      White/Not Hispanic  Aggravated Assault   
1  Black or African American/Not Hispanic  Aggravated Assault   
2                                 Unknown         Destruction   
3                           White/Unknown      Simple Assault   
4                      White/Not Hispanic      Simple Assault   

                     Offense Location                            Bias  \
0                     Park/Playground  Anti-Black or African American   
1  Highway/Road/Alley/Street/Sidewalk                      Anti-White   
2  Highway/Road/Alley/Street/Sidewalk                     Anti-Jewish   
3              Air/Bus/Train Terminal                     Anti-Jewish   
4                      Residence/Home                 Anti-Gay (Male)   

  Victim Type Day of Week  Total Victims  
0  Individual      Sunday              1  
1  Individual   Wednesday              1  
2       Other     Tuesday              0  
3  Individual   Wednesday              0  
4  Individual      Monday              1

In [29]:
#Combine the Offender totals and drop the break out columns
Austin_crime["Total Offenders"] = Austin_crime["Number of Offenders under 18"] + Austin_crime["Number of Offenders over 18"]
Austin_crime = Austin_crime.drop(['Number of Offenders under 18', 'Number of Offenders over 18'], axis=1)
Austin_crime.head()

Month Incident Number Date of Incident/Day of Week  \
0   January     2017-241137                   2017-01-01   
1  February     2017-580344                   2017-02-01   
2     March     2017-800291                   2017-03-21   
3     April    2017-1021534                   2017-04-12   
4       May    2017-1351550                   2017-05-15   

               Race or Ethnic of Offender             Offense  \
0                      White/Not Hispanic  Aggravated Assault   
1  Black or African American/Not Hispanic  Aggravated Assault   
2                                 Unknown         Destruction   
3                           White/Unknown      Simple Assault   
4                      White/Not Hispanic      Simple Assault   

                     Offense Location                            Bias  \
0                     Park/Playground  Anti-Black or African American   
1  Highway/Road/Alley/Street/Sidewalk                      Anti-White   
2  Highway/Road/Alley/Street/Sidewalk                     Anti-Jewish   
3              Air/Bus/Train Terminal                     Anti-Jewish   
4                      Residence/Home                 Anti-Gay (Male)   

  Victim Type Day of Week  Total Victims  Total Offenders  
0  Individual      Sunday              1                1  
1  Individual   Wednesday              1                1  
2       Other     Tuesday              0                0  
3  Individual   Wednesday              0                0  
4  Individual      Monday              1                3

In [30]:
#Create seperate columns for Month, Day, and Year

Austin_crime['Month'] = Austin_crime['Date of Incident/Day of Week'].dt.month
Austin_crime['Day'] = Austin_crime['Date of Incident/Day of Week'].dt.day
Austin_crime['Year'] = Austin_crime['Date of Incident/Day of Week'].dt.year

In [31]:
# View columns
Austin_crime.columns

Index(['Month', 'Incident Number', 'Date of Incident/Day of Week',
       'Race or Ethnic of Offender', 'Offense', 'Offense Location', '    Bias',
       'Victim Type', 'Day of Week', 'Total Victims', 'Total Offenders', 'Day',
       'Year'],
      dtype='object')

In [32]:
# Rename columns
Austin_crime = Austin_crime.rename(columns = {'Incident Number':'Incident_ID', 
                              'Date of Incident/Day of Week':'Incident_Date', 
                              'Race or Ethnic of Offender':'Offender_Race',
                              'Offense':'Offense_Name',
                              'Offense Location':'Offense_Location',
                              '    Bias':'Bias',
                              'Victim Type':'Victim_Type',
                              'Day of Week':'Day_of_Week',
                              'Total Victims':'Total_Victims',
                              'Total Offenders':'Total_Offenders'})
Austin_crime.head()

Month   Incident_ID Incident_Date                           Offender_Race  \
0      1   2017-241137    2017-01-01                      White/Not Hispanic   
1      2   2017-580344    2017-02-01  Black or African American/Not Hispanic   
2      3   2017-800291    2017-03-21                                 Unknown   
3      4  2017-1021534    2017-04-12                           White/Unknown   
4      5  2017-1351550    2017-05-15                      White/Not Hispanic   

         Offense_Name                    Offense_Location  \
0  Aggravated Assault                     Park/Playground   
1  Aggravated Assault  Highway/Road/Alley/Street/Sidewalk   
2         Destruction  Highway/Road/Alley/Street/Sidewalk   
3      Simple Assault              Air/Bus/Train Terminal   
4      Simple Assault                      Residence/Home   

                             Bias Victim_Type Day_of_Week  Total_Victims  \
0  Anti-Black or African American  Individual      Sunday              1   
1                      Anti-White  Individual   Wednesday              1   
2                     Anti-Jewish       Other     Tuesday              0   
3                     Anti-Jewish  Individual   Wednesday              0   
4                 Anti-Gay (Male)  Individual      Monday              1   

   Total_Offenders  Day  Year  
0                1    1  2017  
1                1    1  2017  
2                0   21  2017  
3                0   12  2017  
4                3   15  2017

In [33]:
# Push transformed CSV file to folder
Austin_crime.to_csv("Transformed_Resources/austin_17_18_hatecrime.csv", index=False, header=True)

In [34]:
print(len(Austin_crime.columns))
Austin_crime.columns

13


Index(['Month', 'Incident_ID', 'Incident_Date', 'Offender_Race',
       'Offense_Name', 'Offense_Location', 'Bias', 'Victim_Type',
       'Day_of_Week', 'Total_Victims', 'Total_Offenders', 'Day', 'Year'],
      dtype='object')

In [35]:
#!pip install psycopg2-binary

In [36]:
# Create Postgres connection
rds_connection_string = f'postgres:{SQL_key}@localhost:5432/hate_crimes_db'
engine = create_engine(f'postgresql://{rds_connection_string}')

In [37]:
engine.table_names()

['Texas_Hate_Crimes', 'Austin_Hate_Crimes']

In [38]:
# Push transformed file to Postgres
Austin_crime.to_sql(name='Austin_Hate_Crimes', con=engine, if_exists='append', index=False)

In [39]:
# Query file from Postgres
pd.read_sql_query('select * from "Austin_Hate_Crimes"', con=engine).head()

Incident_ID Incident_Date  Month  Day  Year Day_of_Week  \
0   2017-241137    2017-01-01      1    1  2017      Sunday   
1   2017-580344    2017-02-01      2    1  2017   Wednesday   
2   2017-800291    2017-03-21      3   21  2017     Tuesday   
3  2017-1021534    2017-04-12      4   12  2017   Wednesday   
4  2017-1351550    2017-05-15      5   15  2017      Monday   

         Offense_Name                           Offender_Race  \
0  Aggravated Assault                      White/Not Hispanic   
1  Aggravated Assault  Black or African American/Not Hispanic   
2         Destruction                                 Unknown   
3      Simple Assault                           White/Unknown   
4      Simple Assault                      White/Not Hispanic   

                             Bias Victim_Type  \
0  Anti-Black or African American  Individual   
1                      Anti-White  Individual   
2                     Anti-Jewish       Other   
3                     Anti-Jewish  Individual   
4                 Anti-Gay (Male)  Individual   

                     Offense_Location  Total_Victims  Total_Offenders  
0                     Park/Playground              1                1  
1  Highway/Road/Alley/Street/Sidewalk              1                1  
2  Highway/Road/Alley/Street/Sidewalk              0                0  
3              Air/Bus/Train Terminal              0                0  
4                      Residence/Home              1                3